In [1]:
# Importing all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
reviews_df = pd.read_json(r"C:\Users\manik\Downloads\kindle_reviews\kindle_reviews.json", lines=True)
reviews_df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1F6404F1VG29J,B000F83SZQ,Avidreader,"[0, 0]",I enjoy vintage books and movies so I enjoyed ...,5,Nice vintage story,1399248000,"05 5, 2014"
1,AN0N05A9LIJEQ,B000F83SZQ,critters,"[2, 2]",This book is a reissue of an old one; the auth...,4,Different...,1388966400,"01 6, 2014"
2,A795DMNCJILA6,B000F83SZQ,dot,"[2, 2]",This was a fairly interesting read. It had ol...,4,Oldie,1396569600,"04 4, 2014"
3,A1FV0SX13TWVXQ,B000F83SZQ,"Elaine H. Turley ""Montana Songbird""","[1, 1]",I'd never read any of the Amy Brewster mysteri...,5,I really liked it.,1392768000,"02 19, 2014"
4,A3SPTOKDG7WBLN,B000F83SZQ,Father Dowling Fan,"[0, 1]","If you like period pieces - clothing, lingo, y...",4,Period Mystery,1395187200,"03 19, 2014"


**Data Dictionary**<br>
asin - ID of the product <br>
helpful - helpfulness rating of the review <br>
overall - rating of the product <br>
reviewText - text of the review (heading) <br>
reviewTime - time of the review (raw) <br>
reviewerID - ID of the reviewer <br>
reviewerName - name of the reviewer <br>
summary - summary of the review<br>
unixReviewTime - unix timestamp <br>

In [3]:
unique_items_count = reviews_df['asin'].nunique()
print(unique_items_count)

61934


In [4]:
len(reviews_df)

982619

In [5]:
product_counts = reviews_df["asin"].value_counts()
products_filtered = product_counts[product_counts>=50]

In [6]:
reviews_df_filtered = reviews_df.query("asin in @products_filtered.index.tolist()")

In [7]:
reviews_df_filtered

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
189,A19V8I1LEHA1YW,B000JMLBHU,"Alexander Hollins ""Author, Reader, Editor, Tired""","[0, 0]",A manual on desert island survival in its own ...,4,A classic for a reason,1404000000,"06 29, 2014"
190,A3J5D9XN0QT9AG,B000JMLBHU,Allrie,"[0, 0]",I remember going to the movie theatre when abo...,5,Trouble,1367366400,"05 1, 2013"
191,A33EX8MZYBV0YI,B000JMLBHU,alone with books,"[0, 0]","this was an Ok book, but slow at times, I actu...",3,the Mysterrous Island,1374537600,"07 23, 2013"
192,A3HK1FZVP03IIC,B000JMLBHU,Amazon Customer,"[0, 0]",Classic. What else needs to be said. An advent...,4,Classic,1358208000,"01 15, 2013"
193,A27EEX3H2CB70G,B000JMLBHU,Amazon Customer,"[0, 0]",I read the who thing on my phone. Enjoyed it ...,4,Good Story,1365638400,"04 11, 2013"
...,...,...,...,...,...,...,...,...,...
982567,A20MYFNLY6TGNL,B00LZKMXBI,"The Book Fairy ""The Book Fairy""","[0, 0]",Amazing! I was crying from page 1 And sweating...,5,Tears galore!!!!,1406073600,"07 23, 2014"
982568,A19DWIC1T7127Y,B00LZKMXBI,T,"[3, 3]",I received an advanced reader's copy of this b...,4,Moving On,1405900800,"07 21, 2014"
982569,A28E8S7CY2LJ5W,B00LZKMXBI,Tlh,"[1, 1]","Grab tissues, the first few pages had me cryin...",5,Grab your tissues and enjoy a great story !!!!!,1405900800,"07 21, 2014"
982570,A2GCYN1TPTKX72,B00LZKMXBI,Tricia,"[4, 5]",ARC review..... Let me just start by saying I ...,5,Awesome!!!!!,1405900800,"07 21, 2014"


In the filtered dataframe we are keeping only products that are reviews atleast 50 times

The objective was to build a recommendation system based on the reviews. Now we want to process our dataset, our thoughts was to sample the data from the helpful column which says out of the people who saw the review how many found it helpful, this is our first thought. 

In [8]:
reviews_df["helpful"].iloc[0]

[0, 0]

now I want to check for the ones which are more helpful based on the percentage

In [9]:
reviews_df_filtered["help_percentage"] = reviews_df_filtered["helpful"].apply(lambda x : x[0]/x[1] if x[1]!=0 else 1)

C:\Users\manik\AppData\Local\Temp\ipykernel_19092\3949530801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df_filtered["help_percentage"] = reviews_df_filtered["helpful"].apply(lambda x : x[0]/x[1] if x[1]!=0 else 1)


In [10]:
reviews_df_filtered = reviews_df_filtered.query("help_percentage>=0.8").reset_index(drop =True)

In [11]:
reviews_df_filtered.shape

(222231, 10)

In [12]:
# Now again we filtered those reviews which are really helpful

In [13]:
reviews_df_filtered.reset_index(drop=True, inplace = True)

In [14]:
reviews_df_filtered.isna().sum()

reviewerID            0
asin                  0
reviewerName       2016
helpful               0
reviewText            0
overall               0
summary               0
unixReviewTime        0
reviewTime            0
help_percentage       0
dtype: int64

In [15]:
# Unicode Normalization 
import unicodedata
normalized_text = reviews_df_filtered['reviewText'].apply(lambda x: unicodedata.normalize('NFC', x))
reviews_df_filtered['reviewText'] = normalized_text

In [16]:
# Remove unwanted spaces,extra trailing and leading spaces,punctuations,full-stops,inverted commas,special characters
import re
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) 
    return cleaned_text.strip()
reviews_df_filtered['reviewText'] = reviews_df_filtered['reviewText'].apply(lambda x: clean_text(x))


In [17]:
# Removing stopwords from the dataset
def remove_stopwords(text):
    tokens = word_tokenize(text)    
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
reviews_df_filtered['reviewText'] = reviews_df_filtered['reviewText'].apply(lambda x: remove_stopwords(x))


In [18]:
# Remove unwanted spaces,extra trailing and leading spaces,punctuations,full-stops,inverted commas,special characters
import re
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) 
    return cleaned_text.strip()
reviews_df_filtered['reviewText'] = reviews_df_filtered['reviewText'].apply(lambda x: clean_text(x))


In [19]:
reviews_df_filtered["reviewText"].iloc[0]

'manual desert island survival right book besides old Verne slow build climax love much LOT interesting little asides give lot information attitudes behaviour time treatment castaways towards land around animals Worth reading'

In [27]:
filtered_product = reviews_df_filtered.query("asin=='B000JMLBHU'")

In [24]:
!pip install transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
from transformers import pipeline

# Load sentiment analysis pipeline
classifier = pipeline('sentiment-analysis')


C:\Users\manik\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\manik\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\manik\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [28]:
filtered_product["sentiment_output"] = filtered_product["reviewText"].apply(lambda x: classifier(x))

C:\Users\manik\AppData\Local\Temp\ipykernel_19092\448680821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_product["sentiment_output"] = filtered_product["reviewText"].apply(lambda x: classifier(x))


In [29]:
filtered_product

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,help_percentage,sentiment_output
0,A19V8I1LEHA1YW,B000JMLBHU,"Alexander Hollins ""Author, Reader, Editor, Tired""","[0, 0]",manual desert island survival right book besid...,4,A classic for a reason,1404000000,"06 29, 2014",1.0,"[{'label': 'POSITIVE', 'score': 0.992952167987..."
1,A3J5D9XN0QT9AG,B000JMLBHU,Allrie,"[0, 0]",remember going movie theatre 567 yrs old cousi...,5,Trouble,1367366400,"05 1, 2013",1.0,"[{'label': 'NEGATIVE', 'score': 0.985084950923..."
2,A33EX8MZYBV0YI,B000JMLBHU,alone with books,"[0, 0]",Ok book slow times actually put read something...,3,the Mysterrous Island,1374537600,"07 23, 2013",1.0,"[{'label': 'NEGATIVE', 'score': 0.995464146137..."
3,A3HK1FZVP03IIC,B000JMLBHU,Amazon Customer,"[0, 0]",Classic else needs said adventure classic must...,4,Classic,1358208000,"01 15, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.986055612564..."
4,A27EEX3H2CB70G,B000JMLBHU,Amazon Customer,"[0, 0]",read thing phone Enjoyed lot much King Salomon...,4,Good Story,1365638400,"04 11, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.998269915580..."
...,...,...,...,...,...,...,...,...,...,...,...
99,A1KPDKD7VYRE3B,B000JMLBHU,T. S.,"[1, 1]",Jules Vernes attempt writing Robinson Crusoe S...,4,Verne's version of Robinson Crusoe,1276214400,"06 11, 2010",1.0,"[{'label': 'NEGATIVE', 'score': 0.992653131484..."
100,A258ZUVGV7ZVUH,B000JMLBHU,Wanda ward,"[0, 0]",Good reading anyone age book could shared kids...,5,TheMysterious Island,1360713600,"02 13, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.998368322849..."
101,AAFOJ5PQJ48FO,B000JMLBHU,Wayne,"[0, 0]",Read see non Hollywood version story Gets litt...,3,Where are the Hollywood monsters?,1388448000,"12 31, 2013",1.0,"[{'label': 'NEGATIVE', 'score': 0.995701730251..."
102,A3R19XOP5NA0FK,B000JMLBHU,"Winifred J. Childers ""chillywin""","[0, 0]",liked creative ideas making building items who...,4,mysterious island,1355875200,"12 19, 2012",1.0,"[{'label': 'POSITIVE', 'score': 0.959880530834..."


In [30]:
def extract_label_score(sentiment_dict):
    
    if sentiment_dict and isinstance(sentiment_dict, list):
        return sentiment_dict[0]['label'], sentiment_dict[0]['score']

In [32]:
filtered_product[['label', 'score']] = filtered_product['sentiment_output'].apply(lambda x: pd.Series(extract_label_score(x)))

C:\Users\manik\AppData\Local\Temp\ipykernel_19092\3571133878.py:1: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  filtered_product[['label', 'score']] = filtered_product['sentiment_output'].apply(lambda x: pd.Series(extract_label_score(x)))
C:\Users\manik\AppData\Local\Temp\ipykernel_19092\3571133878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_product[['label', 'score']] = filtered_product['sentiment_output'].apply(lambda x: pd.Series(extract_label_score(x)))
C:\Users\manik\AppData\Local\Temp\ipykernel_19092\3571133878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [34]:
filtered_product.query("label=='POSITIVE'")

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,help_percentage,sentiment_output,label,score
0,A19V8I1LEHA1YW,B000JMLBHU,"Alexander Hollins ""Author, Reader, Editor, Tired""","[0, 0]",manual desert island survival right book besid...,4,A classic for a reason,1404000000,"06 29, 2014",1.0,"[{'label': 'POSITIVE', 'score': 0.992952167987...",POSITIVE,0.992952
3,A3HK1FZVP03IIC,B000JMLBHU,Amazon Customer,"[0, 0]",Classic else needs said adventure classic must...,4,Classic,1358208000,"01 15, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.986055612564...",POSITIVE,0.986056
4,A27EEX3H2CB70G,B000JMLBHU,Amazon Customer,"[0, 0]",read thing phone Enjoyed lot much King Salomon...,4,Good Story,1365638400,"04 11, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.998269915580...",POSITIVE,0.998270
5,A1BKERTS8SPHER,B000JMLBHU,Amazon Customer,"[0, 0]",KEEPS INTERESTED RECOMMEND AGE GENUINE MASTERP...,5,THE MYSTERIOUS ISLAND,1359417600,"01 29, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.999878048896...",POSITIVE,0.999878
7,AB3EF5PVHCBHY,B000JMLBHU,"Amazon Customer ""reader""","[0, 0]",Simply outstanding even though Ive read Verne ...,4,Mysterious Island,1383696000,"11 6, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.999845147132...",POSITIVE,0.999845
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A2V0IOF8CYD9F8,B000JMLBHU,Taylor,"[0, 0]",book interesting fun read characters interesti...,3,Good,1398038400,"04 21, 2014",1.0,"[{'label': 'POSITIVE', 'score': 0.998577833175...",POSITIVE,0.998578
96,A186FFFESJ5JG0,B000JMLBHU,Tezza the terrier,"[0, 0]",excellent read extremely descriptive almost fe...,5,Mysterious Island,1338163200,"05 28, 2012",1.0,"[{'label': 'POSITIVE', 'score': 0.998898506164...",POSITIVE,0.998899
98,A2O0DRN9AU401A,B000JMLBHU,tomkus,"[0, 0]",said book one never read itIt good read nice a...,4,Solid book,1380499200,"09 30, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.986815869808...",POSITIVE,0.986816
100,A258ZUVGV7ZVUH,B000JMLBHU,Wanda ward,"[0, 0]",Good reading anyone age book could shared kids...,5,TheMysterious Island,1360713600,"02 13, 2013",1.0,"[{'label': 'POSITIVE', 'score': 0.998368322849...",POSITIVE,0.998368
